**神经网络的清晰图景**

模型由许多层链接在一起组成，并将输入数据映射为预测值。随后，损失函数将这些预测值与目标值进行比较，得到一个损失值，用于衡量模型预测值与预期结果之间的匹配程度。优化器将利用这个损失值来更新模型权重。
![神经网络、层、损失函数与优化器之间的关系](image/神经网络、层、损失函数与优化器之间的关系.png)

**1、输入数据部分**

``
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255
``

输入图像保存在float32类型的NumPy张量中，其形状分别为(60000, 784)（训练数据）和(10000, 784)（测试数据）。

**2、模型部分**

``
model = keras.Sequential([
    layers.Dense(512, activation="relu"),
    layers.Dense(10, activation="softmax")
])
``

这个模型包含两个链接在一起的Dense层，每层都对输入数据做一些简单的张量运算，这些运算都涉及权重张量。权重张量是该层的属性，里面保存了模型所学到的知识。

**3、模型编译部分**

``
model.compile(optimizer="rmsprop",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
``

sparse_categorical_crossentropy是损失函数，是用于学习权重张量的反馈信号，在训练过程中应使其最小化。降低损失值是通过小批量随机梯度下降来实现的。梯度下降的具体方法由第一个参数给定，即rmsprop优化器。

**4、训练循环部分**

``
model.fit(train_images, train_labels, epochs=5, batch_size=128)
``

在调用fit时发生了什么：模型开始在训练数据上进行迭代（每个小批量包含128个样本），共迭代5轮［在所有训练数据上迭代一次叫作一轮（epoch）］。对于每批数据，模型会计算损失相对于权重的梯度（利用反向传播算法，这一算法源自微积分的链式法则），并将权重沿着减小该批量对应损失值的方向移动。

5轮之后，模型共执行2345次梯度更新（每轮469次），模型损失值将变得足够小，使得模型能够以很高的精度对手写数字进行分类。

## 2.5.1 用TensorFlow从头(不使用Keras的功能)开始重新实现第一个例子

### 简单的Dense类
Dense层实现了下列输入变换，其中W和b是模型参数，activation是一个逐元素的函数（通常是relu，但最后一层是softmax）。

``
output = activation(dot(W, input) + b)
``

In [16]:
# 一个简单的Python类NaiveDense，它创建了两个TensorFlow变量W和b，并定义了一个__call__()方法供外部调用，以实现上述变换
import tensorflow as tf

class NaiveDense:
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        w_shape = (input_size, output_size)  # 创建一个形状为(input_size, output_size)的矩阵W，并将其随机初始化
        w_initial_value = tf.random.uniform(w_shape, minval=0, maxval=1e-1)
        self.W = tf.Variable(w_initial_value)

        b_shape = (output_size,)  # 创建一个形状为(output_size,)的零向量b
        b_initial_value = tf.zeros(b_shape)
        self.b = tf.Variable(b_initial_value)

    def __call__(self, inputs):  # 前向传播
        return self.activation(tf.matmul(inputs, self.W) + self.b)

    @property
    def weights(self):  # 获取该层权重的便捷方法
        return [self.W, self.b]

### 简单的Sequential类
我们创建一个NaiveSequential类，将这些层链接起来。它封装了一个层列表，并定义了一个__call__()方法供外部调用。这个方法将按顺序调用输入的层。它还有一个weights属性，用于记录该层的参数。

In [17]:
class NaiveSequential:
    def __init__(self, layers):
        self.layers = layers

    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x

    @property
    def weights(self):
       weights = []
       for layer in self.layers:
           weights += layer.weights
       return weights

In [18]:
# 利用这个NaiveDense类和NaiveSequential类，我们可以创建一个与Keras类似的模型
model = NaiveSequential([
    NaiveDense(input_size=28 * 28, output_size=512, activation=tf.nn.relu),
    NaiveDense(input_size=512, output_size=10, activation=tf.nn.softmax)
])
assert len(model.weights) == 4

2022-10-15 15:09:55.656884: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### 批量生成器
接下来，需要对MNIST数据进行小批量迭代。

In [19]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size=128):
        assert len(images) == len(labels)
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images) / batch_size)

    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

## 2.5.2　完成一次训练步骤
在一批数据上运行模型后更新模型权重。我们需要做到以下几点。
1. 计算模型对图像批量的预测值。
2. 根据实际标签，计算这些预测值的损失值。
3. 计算损失相对于模型权重的梯度。
4. 将权重沿着梯度的反方向移动一小步。

In [20]:
# 要计算梯度，需要用到TensorFlow GradientTape对象
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:  # (本行及以下4行)运行前向传播，即在GradientTape作用域内计算模型预测值
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(
            labels_batch, predictions)
        average_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(average_loss, model.weights)  # 计算损失相对于权重的梯度。输出gradients是一个列表，每个元素对应model.weights列表中的权重
    update_weights(gradients, model.weights)  # 利用梯度来更新权重（稍后给出这个函数的定义）
    return average_loss

“更新权重”这一步（由update_weights函数实现）的目的，就是将权重沿着减小批量损失值的方向移动“一小步”。移动幅度由学习率决定，它通常是一个很小的数。要实现这个update_weights函数，最简单的方法就是从每个权重中减去gradient * learning_rate。

In [21]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)  # assign_sub相当于TensorFlow变量的-=

在实践中，我们几乎不会像这样手动实现权重更新，而是会使用Keras的Optimizer实例，如下所示：

``
from tensorflow.keras import optimizers
optimizer = optimizers.SGD(learning_rate=1e-3)
def update_weights(gradients, weights):
    optimizer.apply_gradients(zip(gradients, weights))
``

在我们实现了对每批数据的训练后，下面继续实现一轮完整的训练

## 2.5.3完整的训练循环
一轮训练就是对训练数据的每个批量都重复上述训练步骤，而完整的训练循环就是重复多轮训练。

In [22]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch {epoch_counter}")
        batch_generator = BatchGenerator(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch {batch_counter}: {loss:.2f}")

试运行

In [23]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255

fit(model, train_images, train_labels, epochs=10, batch_size=128)

Epoch 0
loss at batch 0: 4.10
loss at batch 100: 2.25
loss at batch 200: 2.21
loss at batch 300: 2.10
loss at batch 400: 2.24
Epoch 1
loss at batch 0: 1.93
loss at batch 100: 1.90
loss at batch 200: 1.84
loss at batch 300: 1.72
loss at batch 400: 1.85
Epoch 2
loss at batch 0: 1.60
loss at batch 100: 1.60
loss at batch 200: 1.52
loss at batch 300: 1.44
loss at batch 400: 1.52
Epoch 3
loss at batch 0: 1.34
loss at batch 100: 1.35
loss at batch 200: 1.25
loss at batch 300: 1.23
loss at batch 400: 1.28
Epoch 4
loss at batch 0: 1.14
loss at batch 100: 1.17
loss at batch 200: 1.05
loss at batch 300: 1.06
loss at batch 400: 1.11
Epoch 5
loss at batch 0: 0.99
loss at batch 100: 1.03
loss at batch 200: 0.91
loss at batch 300: 0.94
loss at batch 400: 0.99
Epoch 6
loss at batch 0: 0.88
loss at batch 100: 0.92
loss at batch 200: 0.80
loss at batch 300: 0.85
loss at batch 400: 0.90
Epoch 7
loss at batch 0: 0.80
loss at batch 100: 0.84
loss at batch 200: 0.72
loss at batch 300: 0.78
loss at batch 40

## 评估模型
评估模型，方法是对模型在测试图像上的预测值取argmax，并将其与预期标签进行比较。

In [29]:
import numpy as np
predictions = model(test_images)
predictions = predictions.numpy() # 对TensorFlow张量调用.numpy()，可以将其转换为Numpy张量
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f}")

accuracy: 0.81


可以看到，用几行Keras代码就能完成的工作，手动实现起来还是挺费劲的。但手动实现一遍之后，应该能够清楚地了解在调用fit()时神经网络内部都发生了什么。拥有这种对代码底层原理的思维模型，可以让你更好地使用Keras API的高级功能。